In [1]:
from PySW import *
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

In [2]:
omega = RDSymbol('omega', positive=True, real=True)
omega_z = RDSymbol('omega_z', positive=True, real=True)
g = RDSymbol('g', order=1, positive=True, real=True)

E0 = RDSymbol('E0', order=1, positive=True, real=True)
omega_d = RDSymbol('omega_d', positive=True, real=True)

a = BosonOp('a')
ad = Dagger(a)

Spin = RDBasis('sigma', dim=2)
s0, sx, sy, sz = Spin.basis

H0 = omega * ad * a + sp.Rational(1, 2) * omega_z * sz
H0_mat = H0.subs(ad*a, ad*a*s0.matrix).subs(sz, sz.matrix)
V = g * (ad + a) * sx
V_mat = V.subs(sx, sx.matrix)
Hd = E0 * sp.exp(sp.I * omega_d * t) * a + E0 * sp.exp(-sp.I * omega_d * t) * ad
Hd_mat = Hd.subs(a, a*s0.matrix).subs(ad, ad*s0.matrix)

display(H0+V+Hd)

E0*exp(I*omega_d*t)*a + E0*exp(-I*omega_d*t)*Dagger(a) + g*(Dagger(a) + a)*sigma_1 + omega*Dagger(a)*a + omega_z*sigma_3/2

In [3]:
def omega_uu(N, u):
    state_u = sp.Matrix([1 , 0]) if u == 0 else sp.Matrix([0 , 1])
    E = state_u.T * H0.subs(ad*a, N * s0.matrix).subs(sz, sz.matrix) * state_u
    return E[0]

def w_uu_delta(u, v, delta):
    return (omega_uu(ad*a + delta, v) - omega_uu(ad*a, u)).cancel()

o01_1 = w_uu_delta(0, 1, 1)
o10_1 = w_uu_delta(1, 0, 1)
o00_1 = w_uu_delta(0, 0, 1)
o11_1 = w_uu_delta(1, 1, 1)

display_dict({
    sp.Symbol('\\omega_{01}^{(1)}'): o01_1,
    sp.Symbol('\\omega_{10}^{(1)}'): o10_1,
    sp.Symbol('\\omega_{00}^{(1)}'): o00_1,
    sp.Symbol('\\omega_{11}^{(1)}'): o11_1,
})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [4]:
gamma01_1 = - (-g)/(o01_1)
gamma10_1 = - (-g)/(o10_1)
gamma00_1 = - (-E0 * sp.exp(sp.I * omega_d * t))/(o00_1 - hbar * omega_d)
gamma11_1 = - (-E0 * sp.exp(sp.I * omega_d * t))/(o11_1 - hbar * omega_d)

s01 = sp.Rational(1,2) * (sx.matrix + sp.I * sy.matrix)
s10 = sp.Rational(1,2) * (sx.matrix - sp.I * sy.matrix)
s00 = sp.Rational(1,2) * (s0.matrix + sz.matrix)    
s11 = sp.Rational(1,2) * (s0.matrix - sz.matrix)

S_for_V  = gamma01_1 * a * s01 + gamma10_1 * a * s10
S_for_Vd = gamma01_1 * ad * s01.T + gamma10_1 * ad * s10.T
S_V = S_for_V - S_for_Vd

S_for_Hd = gamma00_1 * a * s00 + gamma11_1 * a * s11
S_for_Hdd = gamma00_1.conjugate() * ad * s00.T + gamma11_1.conjugate() * ad * s11.T
S_Hd = S_for_Hd - S_for_Hdd

In [5]:
def apply_subs(sympy_expr, subs_dict):
    prev_expr = sympy_expr
    sympy_expr = sympy_expr.subs(subs_dict).expand()
    while prev_expr != sympy_expr:
        prev_expr = sympy_expr
        sympy_expr = sympy_expr.subs(subs_dict).expand()
    return sympy_expr

S = S_V + S_Hd
apply_subs((H0_mat*S - S*H0_mat + V_mat + Hd_mat - sp.I * hbar * sp.diff(S, t)).expand(), {a*ad: ad*a + 1}).cancel()

Matrix([
[0, 0],
[0, 0]])

In [6]:
S

Matrix([
[E0*exp(I*omega_d*t)*a/(-hbar*omega_d + omega) - E0*exp(-I*omega_d*t)*Dagger(a)/(-hbar*omega_d + omega),                                                 -g*Dagger(a)/(omega + omega_z) + g*a/(omega - omega_z)],
[                                                 g*a/(omega + omega_z) - g*Dagger(a)/(omega - omega_z), E0*exp(I*omega_d*t)*a/(-hbar*omega_d + omega) - E0*exp(-I*omega_d*t)*Dagger(a)/(-hbar*omega_d + omega)]])

In [ ]:
S = S_V + S_Hd
V1 = V_mat + Hd_mat

H_eff = H0_mat + sp.Rational(1, 2) * (V1 * S - S * V1) -  sp.I * hbar * (sp.diff(S, t) * S - S * sp.diff(S, t))
res = Spin.project(apply_subs(H_eff.expand(), {a*ad: ad*a + 1})).expand()
res = group_by_operators(res)
res = {key: sp.expand_complex(sp.simplify(val)).trigsimp().factor() for key, val in res.items()}
display_dict(res)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [28]:
res = Spin.project(apply_subs(- sp.Rational(1, 2) * sp.I * hbar * (sp.diff(S, t) * S - S * sp.diff(S, t)), {a*ad: ad*a + 1})).expand()
res = group_by_operators(res)
res = {key: sp.expand_complex(sp.simplify(val)).trigsimp().factor() for key, val in res.items()}
display_dict(res)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [29]:
sp.diff(S, t)

Matrix([
[I*E0*omega_d*exp(I*omega_d*t)*a/(-hbar*omega_d + omega) + I*E0*omega_d*exp(-I*omega_d*t)*Dagger(a)/(-hbar*omega_d + omega),                                                                                                                          0],
[                                                                                                                         0, I*E0*omega_d*exp(I*omega_d*t)*a/(-hbar*omega_d + omega) + I*E0*omega_d*exp(-I*omega_d*t)*Dagger(a)/(-hbar*omega_d + omega)]])

In [79]:
- sp.Rational(1, 2) * sp.I * hbar * (sp.diff(S, t) * S - S * sp.diff(S, t))

Matrix([
[-I*hbar*(-(E0*exp(I*omega_d*t)*a/(-hbar*omega_d + omega) - E0*exp(-I*omega_d*t)*Dagger(a)/(-hbar*omega_d + omega))*(I*E0*omega_d*exp(I*omega_d*t)*a/(-hbar*omega_d + omega) + I*E0*omega_d*exp(-I*omega_d*t)*Dagger(a)/(-hbar*omega_d + omega)) + (I*E0*omega_d*exp(I*omega_d*t)*a/(-hbar*omega_d + omega) + I*E0*omega_d*exp(-I*omega_d*t)*Dagger(a)/(-hbar*omega_d + omega))*(E0*exp(I*omega_d*t)*a/(-hbar*omega_d + omega) - E0*exp(-I*omega_d*t)*Dagger(a)/(-hbar*omega_d + omega)))/2,                                                                                                 -I*hbar*(-(-g*Dagger(a)/(omega + omega_z) + g*a/(omega - omega_z))*(I*E0*omega_d*exp(I*omega_d*t)*a/(-hbar*omega_d + omega) + I*E0*omega_d*exp(-I*omega_d*t)*Dagger(a)/(-hbar*omega_d + omega)) + (I*E0*omega_d*exp(I*omega_d*t)*a/(-hbar*omega_d + omega) + I*E0*omega_d*exp(-I*omega_d*t)*Dagger(a)/(-hbar*omega_d + omega))*(-g*Dagger(a)/(omega + omega_z) + g*a/(omega - omega_z)))/2],
[                                    

In [ ]:
display_dict(group_by_operators(Spin.project(sp.Matrix([
    [0, sp.exp(-sp.I*omega)],
    [sp.exp(sp.I*omega), 0]

])).expand().subs(sp.exp(sp.I*omega), sp.cos(omega) + sp.I * sp.sin(omega)).factor()))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [76]:
A = H0_mat * S - S * H0_mat
sp.Rational(1,2) * sp.factor_terms(apply_subs((A * S - S * A ).expand(), {a*ad: ad*a + 1}).subs(a,0).subs(ad,0))

Matrix([
[                                                                                                                                                                                                                                  E0**2*omega/(hbar**2*omega_d**2 - 2*hbar*omega*omega_d + omega**2) + g**2*omega/(omega**2 - 2*omega*omega_z + omega_z**2) - g**2*omega_z/(omega**2 - 2*omega*omega_z + omega_z**2), E0*g*(2*omega*exp(-I*omega_d*t)/(-hbar*omega*omega_d + hbar*omega_d*omega_z + omega**2 - omega*omega_z) + 2*omega*exp(I*omega_d*t)/(-hbar*omega*omega_d - hbar*omega_d*omega_z + omega**2 + omega*omega_z) - omega_z*exp(-I*omega_d*t)/(-hbar*omega*omega_d + hbar*omega_d*omega_z + omega**2 - omega*omega_z) + omega_z*exp(I*omega_d*t)/(-hbar*omega*omega_d - hbar*omega_d*omega_z + omega**2 + omega*omega_z))/2],
[E0*g*(2*omega*exp(I*omega_d*t)/(-hbar*omega*omega_d + hbar*omega_d*omega_z + omega**2 - omega*omega_z) + 2*omega*exp(-I*omega_d*t)/(-hbar*omega*omega_d - hbar*omega_d*omega_z +